### Задание
Создайте FastAPI-приложение, реализующее работу простого калькулятора.
Приложение должно реализовывать следующие методы:

1. Методы сложения, вычитания, умножения, деления.
2. Методы создания выражения, возможность добавления сложных выражений, таких как, (а+b)* c + (d - e)/(f-g). Пример: отдельный метод, который принимает аргументы a, op и b; где a, b — атрибуты. op — операция над ними.
Нужно так же реализовать возможность использования выражения в составе более сложного выражения. Пример: метод принимает строку вида (а+b)* c + (d - e)/(f-g). Бизнес-логика парсит строку, расставляет правильным образом порядок выполнения операций, производит вычисление и возвращает ответ. В этом случае потребность в выполнение пунктов 3 и 4 отпадает.
3. Метод просмотра текущего выражения. Метод возвращает состояние выражения сформированное на текущий момент.
4. Метод выполнения выражения. Метод возвращает результат выполнения выражения.

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import operator as op

app = FastAPI(title="Calculator API")

# ---------- 1) Базовые операции через /op ----------

class OpInput(BaseModel):
    a: float
    op: str
    b: float

ops = {"+": op.add,
    "-": op.sub,
    "*": op.mul,
    "/": op.truediv}

def apply_op(a: float, oper: str, b: float) -> float:
    func = ops.get(oper)
    if not func:
        raise HTTPException(status_code=400, detail="op должен быть одним из математических символов: + - * /")
    if oper == "/" and b == 0:
        raise HTTPException(status_code=400, detail="Деление на ноль не определено")
    return func(a, b)

def pretty_number(x: float):
    return int(x) if float(x).is_integer() else x

@app.post("/op")
def op_simple(body: OpInput):
    res = apply_op(body.a, body.op, body.b)
    return {"result": pretty_number(res)}



# ---------- 2) Составные выражения одной строкой через /calc ----------

class Expr(BaseModel):
    expression: str

def evaluate(s: str) -> float:
    i = 0

    def peek() -> str:
        # вернуть текущий символ или пустую строку, если дошли до конца
        return s[i] if i < len(s) else ""

    def eat_ws():
        # пропустить все пробелы
        nonlocal i
        while i < len(s) and s[i].isspace():
            i += 1

    def eat(ch: str) -> bool:
        # если следующий символ равен ch — «съесть» его и вернуть True
        nonlocal i
        eat_ws()
        if peek() == ch:
            i += 1
            return True
        return False

    def parse_number() -> float:
        # читаем число: последовательность цифр и максимум одна точка
        nonlocal i
        eat_ws()
        start = i
        dot_used = False
        while i < len(s):
            c = s[i]
            if c.isdigit():
                i += 1
            elif c == "." and not dot_used:
                dot_used = True
                i += 1
            else:
                break
        if i == start:  # ничего не прочитали
            raise HTTPException(status_code=400, detail="Ожидалось число")
        try:
            return float(s[start:i])
        except ValueError:
            raise HTTPException(status_code=400, detail="Некорректное число")

    def parse_factor() -> float:
        eat_ws()
        if eat("+"):
            return parse_factor()
        if eat("-"):
            return -parse_factor()
        if eat("("):
            val = parse_expr()
            if not eat(")"):
                raise HTTPException(status_code=400, detail="Нет закрывающей скобки")
            return val
        return parse_number()

    def parse_term() -> float:
        val = parse_factor()
        while True:
            eat_ws()
            if eat("*"):
                val *= parse_factor()
            elif eat("/"):
                denom = parse_factor()
                if denom == 0:
                    raise HTTPException(status_code=400, detail="Деление на ноль")
                val /= denom
            else:
                break
        return val

    def parse_expr() -> float:
        val = parse_term()
        while True:
            eat_ws()
            if eat("+"):
                val += parse_term()
            elif eat("-"):
                val -= parse_term()
            else:
                break
        return val

    result = parse_expr()
    eat_ws()
    if i != len(s):
        raise HTTPException(status_code=400, detail="Лишние символы в конце выражения")
    return result

@app.post("/calc")
def calc(body: Expr):
    return {"result": evaluate(body.expression)}
